In [1]:
import pandas as pd
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [3]:
# Load the flight prices from Excel
flight_data = pd.read_excel("RandomFlightPrices.xlsx")

In [4]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Today's date is October 4, 2024. For any date calculations, use this. Assume year is 2024 if not mentioned"
system_message += "Always be accurate. If you don't know the answer, say so."

In [5]:
def find_cheapest_route(from_city, to_city, date, max_stops=2, current_route=None, visited=None):
    if visited is None:
        visited = set()
    if current_route is None:
        current_route = []
    
    # Mark this city as visited to avoid infinite loops
    visited.add(from_city)

    # Check for direct flights
    direct_flight = flight_data[
        (flight_data['From'].str.lower() == from_city.lower()) &
        (flight_data['To'].str.lower() == to_city.lower()) &
        (flight_data['Date'] == date)
    ]
    
    if not direct_flight.empty:
        min_price = direct_flight['Min Price'].min()
        return {"route": current_route + [(from_city, to_city)], "price": min_price}
    
    if max_stops == 0:
        return None  # No more stops allowed

    # Search for connecting flights
    connections = flight_data[
        (flight_data['From'].str.lower() == from_city.lower()) &
        (~flight_data['To'].str.lower().isin(visited)) &
        (flight_data['Date'] == date)
    ]

    cheapest_route = None

    for _, flight in connections.iterrows():
        next_city = flight['To']
        price = flight['Min Price']

        # Recursively search for flights from this next city to the final destination
        result = find_cheapest_route(next_city, to_city, date, max_stops - 1, current_route + [(from_city, next_city)], visited.copy())
        
        if result:
            total_price = price + result["price"]
            if cheapest_route is None or total_price < cheapest_route["price"]:
                cheapest_route = {"route": result["route"], "price": total_price}

    return cheapest_route


In [7]:
# Function to get the cheapest flight, checking for multi-leg routes
def get_cheapest_flight(from_city, to_city, date):
    result = find_cheapest_route(from_city, to_city, date, max_stops=3)  # Allow up to 3 stops
    if result:
        route = " → ".join([f"{from_}-{to_}" for from_, to_ in result["route"]])
        return f"The cheapest route from {from_city} to {to_city} on {date} is: {route}, costing ${result['price']}"
    else:
        return f"No routes found from {from_city} to {to_city} on {date}."

In [8]:
get_cheapest_flight('dallas', 'kochi', '10/12/2024')

'The cheapest route from dallas to kochi on 10/12/2024 is: dallas-London → London-Qatar → Qatar-kochi, costing $372'

## List All Flights on a Given Day
We can write a function that returns all flights available on a particular day. This function will filter the flight data for a given date and return a list of all possible routes with their prices.

In [14]:
def list_all_flights_on_date(from_city, to_city, date):
    flights = flight_data[
        (flight_data['From'].str.lower() == from_city.lower()) &
        (flight_data['To'].str.lower() == to_city.lower()) &
        (flight_data['Date'] == date)
    ]
    
    if flights.empty:
        return f"No flights available from {from_city} to {to_city} on {date}."
    
    flight_list = []
    for _, flight in flights.iterrows():
        flight_list.append(f"Flight from {flight['From']} to {flight['To']} costs ${flight['Min Price']}.")
    
    return "Available flights: \n" + "\n".join(flight_list)


## Find the Cheapest Flight Around a Date Range
For this, we can extend the query to search for the cheapest flight within a range of dates (e.g., 5 days around a given date).

In [15]:
def find_cheapest_flight_around_date(from_city, to_city, base_date, date_range=2):
    # Convert base date to datetime for easier manipulation
    base_date = pd.to_datetime(base_date)
    start_date = base_date - pd.Timedelta(days=date_range)
    end_date = base_date + pd.Timedelta(days=date_range)
    
    flights = flight_data[
        (flight_data['From'].str.lower() == from_city.lower()) &
        (flight_data['To'].str.lower() == to_city.lower()) &
        (flight_data['Date'] >= start_date) &
        (flight_data['Date'] <= end_date)
    ]
    
    if flights.empty:
        return f"No flights available from {from_city} to {to_city} between {start_date.date()} and {end_date.date()}."
    
    cheapest_flight = flights.loc[flights['Min Price'].idxmin()]
    return (f"The cheapest flight from {from_city} to {to_city} between {start_date.date()} and {end_date.date()} "
            f"is on {cheapest_flight['Date'].date()} with a price of ${cheapest_flight['Min Price']}.")


#### price_function is a dictionary structure 
that describes the get_ticket_price function. This dictionary includes metadata such as the function's name, a description of what it does, and the parameters it expects.
The tools list holds this function description to allow the assistant to use it when a user asks about ticket prices.

In [9]:
# Define the tool to describe the function
price_function = {
    "name": "get_cheapest_flight",
    "description": "Get the cheapest price for a return ticket from one city to another, considering layovers if necessary.",
    "parameters": {
        "type": "object",
        "properties": {
            "from_city": {
                "type": "string",
                "description": "The city the customer wants to depart from",
            },
            "to_city": {
                "type": "string",
                "description": "The destination city the customer wants to travel to",
            },
            "date": {
                "type": "string",
                "description": "The date of the flight in YYYY-MM-DD format",
            },
        },
        "required": ["from_city", "to_city", "date"],
        "additionalProperties": False
    }
}

In [10]:
# Include the tool in the list of tools
tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use our Tool
There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool. If the model determines that a tool should be called, the function handle_tool_call is triggered.

Here's how the function looks:

In [27]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": message})
    
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response, (from_city, to_city, date) = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [28]:
# Handle tool calls
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    from_city = arguments.get('from_city')
    to_city = arguments.get('to_city')
    date = arguments.get('date')
    
    price = get_cheapest_flight(from_city, to_city, date)
    
    response = {
        "role": "tool",
        "content": json.dumps({"from_city": from_city, "to_city": to_city, "date": date, "price": price}),
        "tool_call_id": message.tool_calls[0].id
    }
    return response, (from_city, to_city, date)

In [29]:
gr.ChatInterface(fn=chat).launch(share=True)

Running on local URL:  http://127.0.0.1:7897
Running on public URL: https://8ac5652ca8d4191fe0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
